# Activity: Validate and clean your data

## Introduction

In this activity, you will use input validation and label encoding to prepare a dataset for analysis. These are fundamental techniques used in all types of data analysis, from simple linear regression to complex neural networks. 

In this activity, you are a data professional an investment firm that is attempting to invest in private companies with a valuation of at least $1 billion. These are often known as "unicorns." Your client wants to develop a better understanding of unicorns, with the hope they can be early investors in future highly successful companies. They are particularly interested in the investment strategies of the three top unicorn investors: Sequoia Capital, Tiger Global Management, and Accel. 

## Step 1: Imports 

Import relevant Python libraries and packages: `numpy`, `pandas`, `seaborn`, and `pyplot` from `matplotlib`.

In [30]:
# Import libraries and packages.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


### Load the dataset

The data contains details about unicorn companies, such as when they were founded, when they achieved unicorn status, and their current valuation. The dataset `Modified_Unicorn_Companies.csv` is loaded as `companies`, now display the first five rows. The variables in the dataset have been adjusted to suit the objectives of this lab, so they may be different from similar data used in prior labs.

In [31]:
# Run this cell so pandas displays all columns
pd.set_option('display.max_columns', None)

In [32]:
# RUN THIS CELL TO IMPORT YOUR DATA. 
companies = pd.read_csv(r'C:\Users\saswa\Documents\GitHub\Python-For-Data-Analysis\Course-3\Data\Module_3\Modified_Unicorn_Companies.csv')

# Display the first five rows.
companies.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S..."
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China..."
3,Stripe,95,2014-01-23,FinTech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG"
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita..."


## Step 2: Data cleaning


Begin by displaying the data types of the columns in `companies`.

In [33]:
# Display the data types of the columns.

companies.dtypes

Company             object
Valuation            int64
Date Joined         object
Industry            object
City                object
Country/Region      object
Continent           object
Year Founded         int64
Funding             object
Select Investors    object
dtype: object

### Modify the data types

Notice that the data type of the `Date Joined` column is an `object`&mdash;in this case, a string. Convert this column to `datetime` to make it more usable. 

In [34]:
# Apply necessary datatype conversions.

companies['Date Joined']=pd.to_datetime(companies['Date Joined'])

### Create a new column

Add a column called `Years To Unicorn`, which is the number of years between when the company was founded and when it became a unicorn.

In [35]:
# Create the column Years To Unicorn.

companies['Years To Unicorn']=companies['Date Joined'].dt.year-companies['Year Founded']

**QUESTION: Why might your client be interested in how quickly a company achieved unicorn status?**

I believe my client would be interested in how quickly a company achieves unicorn status because it can provide valuable insights into the company’s growth and potential for future success. If a company reaches unicorn status in a short period, it indicates rapid scaling and a high valuation, which often suggests strong performance, innovation, or high market demand. This could signal a promising investment opportunity or at least justify further investigation into the company’s long-term potential. By identifying companies that achieve unicorn status quickly, my client could uncover high-potential investment opportunities.

### Input validation

The data has some issues with bad data, duplicate rows, and inconsistent `Industry` labels.

Identify and correct each of these issues.

#### Correcting bad data

Get descriptive statistics for the `Years To Unicorn` column.

In [36]:
# Identify and correct the issue with Years To Unicorn.

companies['Years To Unicorn'].describe()

count    1074.000000
mean        7.013035
std         5.331842
min        -3.000000
25%         4.000000
50%         6.000000
75%         9.000000
max        98.000000
Name: Years To Unicorn, dtype: float64

Isolate all rows where the `Years To Unicorn` column contains a negative value.

In [37]:
# Isolate any rows where `Years To Unicorn` is negative

companies[companies['Years To Unicorn']<0]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn
527,InVision,2,2017-11-01,Internet software & services,New York,United States,North America,2020,$349M,"FirstMark Capital, Tiger Global Management, IC...",-3


**Question: How many rows have negative values in the `Years To Unicorn` column, and what companies are they for?**


There is only one row with a negative value in the Years To Unicorn column, corresponding to observation 527. This row represents the company InVision.

An internet search reveals that InVision was founded in 2011. Replace the value at `Year Founded` with `2011` for InVision's row. 

In [38]:
# Replace InVision's `Year Founded` value with 2011

companies.loc[companies['Company']=='InVision','Year Founded']=2011

# Verify the change was made properly

companies[companies['Company']=='InVision']

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn
527,InVision,2,2017-11-01,Internet software & services,New York,United States,North America,2011,$349M,"FirstMark Capital, Tiger Global Management, IC...",-3


Now, recalculate all the values in the `Years To Unicorn` column to remove the negative value for InVision. Verify that there are no more negative values afterwards.

In [39]:
# Recalculate all values in the `Years To Unicorn` column

companies['Years To Unicorn'] = companies['Date Joined'].dt.year - companies['Year Founded']

# Verify that there are no more negative values in the column

companies['Years To Unicorn'].describe()

count    1074.000000
mean        7.021415
std         5.323155
min         0.000000
25%         4.000000
50%         6.000000
75%         9.000000
max        98.000000
Name: Years To Unicorn, dtype: float64

#### Issues with `Industry` labels

The company provided you with the following list of industry labels to identify in the data for `Industry`. 

**Note:** Any labels in the `Industry` column that are not in `industry_list` are misspellings.

In [40]:
# List provided by the company of the expected industry labels in the data
industry_list = ['Artificial intelligence', 'Other','E-commerce & direct-to-consumer', 'Fintech',\
       'Internet software & services','Supply chain, logistics, & delivery', 'Consumer & retail',\
       'Data management & analytics', 'Edtech', 'Health', 'Hardware','Auto & transportation', \
        'Travel', 'Cybersecurity','Mobile & telecommunications']

First, check if there are values in the `Industry` column that are not in `industry_list`. If so, what are they?

In [41]:
# Check which values are in `Industry` but not in `industry_list`

set(companies['Industry'])-set(industry_list)

{'Artificial Intelligence', 'Data management and analytics', 'FinTech'}

**Question: Which values currently exist in the `Industry` column that are not in `industry_list`?**



The values currently present in the Industry column but not in the industry_list are misspellings: 'Artificial Intelligence', 'Data management and analytics', and 'FinTech'.

Now, correct the bad entries in the `Industry` column by replacing them with an approved string from `industry_list`. To do this, use the [`replace()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.replace.html) `Series` method on the `Industry` series. When you pass a dictionary to the method, it will replace the data in the series where that data matches the dictionary's keys. The values that get imputed are the values of the dictionary. If a value is not specified in the dictionary, the series' original value is retained.
</br></br>


```
Example:

 [IN]: column_a = pd.Series(['A', 'B', 'C', 'D'])
       column_a

[OUT]: 0    A
       1    B
       2    C
       3    D
       dtype: object

 [IN]: replacement_dict = {'A':'z', 'B':'y', 'C':'x'}
       column_a = column_a.replace(replacement_dict)
       column_a
       
[OUT]: 0    z
       1    y
       2    x
       3    D
       dtype: object

```

1. Create a dictionary called `replacement_dict` whose keys are the incorrect spellings in the `Industry` series and whose values are the correct spelling, as indicated in `industry_list`.

2. Call the `replace()` method on the `Industry` series and pass to it `replacement_dict` as its argument. Reassign the result back to the `Industry` column.

3. Verify that there are no longer any elements in `Industry` that are not in `industry_list`.

In [42]:
# 1. Create `replacement_dict`

replacement_dict={'Artificial Intelligence':'Artificial intelligence',
                  'Data management and analytics':'Data management & analytics',
                  'FinTech':'Fintech'}

# 2. Replace the incorrect values in the `Industry` column

companies['Industry']=companies['Industry'].replace(replacement_dict)

# 3. Verify that there are no longer any elements in `Industry` that are not in `industry_list`

set(companies['Industry'])-set(industry_list)

set()

#### Handling duplicate rows

The business mentioned that no company should appear in the data more than once.

Verify that this is indeed the case, and if not, clean the data so each company appears only once.

Begin by checking which, if any, companies are duplicated. Filter the data to return all occurrences of those duplicated companies.

In [43]:
# Isolate rows of all companies that have duplicates

companies[companies.duplicated(subset=['Company'], keep=False)]



,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn
385,BrewDog,2,2017-04-10,Consumer & retail,Aberdeen,United Kingdom,Europe,2007,$233M,"TSG Consumer Partners, Crowdcube",10
386,BrewDog,2,2017-04-10,Consumer & retail,Aberdeen,UnitedKingdom,Europe,2007,$233M,TSG Consumer Partners,10
510,ZocDoc,2,2015-08-20,Health,New York,United States,North America,2007,$374M,"Founders Fund, Khosla Ventures, Goldman Sachs",8
511,ZocDoc,2,2015-08-20,Health,NaN,United States,North America,2007,$374M,Founders Fund,8
1031,SoundHound,1,2018-05-03,Artificial intelligence,Santa Clara,United States,North America,2005,$215M,"Tencent Holdings, Walden Venture Capital, Glob...",13
1032,SoundHound,1,2018-05-03,Other,Santa Clara,United States,North America,2005,$215M,Tencent Holdings,13


**Question: Do these duplicated companies seem like legitimate data points, or are they problematic data?**


The duplicated companies don't seem legitimate. They appear to be repeated entries of the same company with minor variations, rather than different companies with the same name.

Keep the first occurrence of each duplicate company and drop the subsequent rows that are copies.

In [44]:
# Drop rows of duplicate companies after their first occurrence

companies=companies.drop_duplicates(subset=['Company'],keep='first')

**Question: Why is it important to perform input validation?**


Input validation is an important practice because it ensures that data is complete, consistent, and of high quality. It is an iterative process that can be performed at any stage of the exploratory data analysis (EDA) or even between stages. Regular validation is necessary to catch anomalies, ensure data integrity, and maintain dependability throughout the project. This process helps identify and correct errors, missing values, or inconsistencies, preventing incorrect or misleading analysis. Additionally, input validation safeguards the data against any unauthorized tampering during analysis or other tasks within the workflow.

**Question: What steps did you take to perform input validation for this dataset?**


To perform input validation for this dataset, I took the following steps:

- First, I converted the Date Joined column from an object dtype to a datetime object, ensuring the dates were in the correct format and enabling date-related operations.
- I also ensured that the Years To Unicorn field did not contain any negative values, as negative values would imply anomalous insights, such as a company achieving unicorn status before being founded.
- I then reviewed the Industry column for any discrepancies between the values in the original dataset and the approved list provided by the company. - I corrected any text casing differences and replaced the incorrect values with the appropriate entries from the list.
- Lastly, I detected and removed duplicate entries based on the Company field, retaining only the first occurrence, which I considered the original.

### Convert numerical data to categorical data

Sometimes, you'll want to simplify a numeric column by converting it to a categorical column. To do this, one common approach is to break the range of possible values into a defined number of equally sized bins and assign each bin a name. In the next step, you'll practice this process.

#### Create a `High Valuation` column

The data in the `Valuation` column represents how much money (in billions, USD) each company is valued at. Use the `Valuation` column to create a new column called `High Valuation`. For each company, the value in this column should be `low` if the company is in the bottom 50% of company valuations and `high` if the company is in the top 50%.

In [45]:
pd.options.mode.copy_on_write = True


In [46]:
# Create new `High Valuation` column
# Use qcut to divide Valuation into 'high' and 'low' Valuation groups

companies['High Valuation']=pd.qcut(companies['Valuation'],q=2,labels=['low','high'])

### Convert categorical data to numerical data

Three common methods for changing categorical data to numerical are:

1. Label encoding: order matters (ordinal numeric labels)
2. Label encoding: order doesn't matter (nominal numeric labels)
3. Dummy encoding: order doesn't matter (creation of binary columns for each possible category contained in the variable)

The decision on which method to use depends on the context and must be made on a case-to-case basis. However, a distinction is typically made between categorical variables with equal weight given to all possible categories vs. variables with a hierarchical structure of importance to their possible categories.  

For example, a variable called `subject` might have possible values of `history`, `mathematics`, `literature`. In this case, each subject might be **nominal**&mdash;given the same level of importance. However, you might have another variable called `class`, whose possible values are `freshman`, `sophomore`, `junior`, `senior`. In this case, the class variable is **ordinal**&mdash;its values have an ordered, hierarchical structure of importance. 

Machine learning models typically need all data to be numeric, and they generally use ordinal label encoding (method 1) and dummy encoding (method 3). 

In the next steps, you'll convert the following variables: `Continent`, `Country/Region`, and `Industry`, each using a different approach.

When to Use Which:
Use ordinal or nominal label encoding (cat.codes) when numeric representation is required:

cat.codes can be used for both ordinal and nominal data. For ordinal data, it assigns numeric codes based on the order of categories (e.g., Low, Medium, High). For nominal data, it assigns unique numeric codes without implying any meaningful order. However, note that the order of categories might still influence the numeric labels assigned, and this depends on how the machine learning model interprets the data. Some models may treat these numeric labels as a meaningful ordinal relationship (like in linear models), while others (like tree-based models) may interpret them as distinct categories without implying order.
Use nominal label encoding (LabelEncoder) when the order doesn't matter but numeric representation is required:

LabelEncoder is typically used for nominal data, where categories have no inherent order (e.g., Country, Region). It assigns unique numeric labels to each category without implying any ranking or hierarchy.
Use dummy encoding (get_dummies() or OneHotEncoder) when the order doesn't matter and you want to avoid introducing unnecessary ordinal relationships:

Dummy encoding creates separate binary columns for each category, ensuring that no ordinal relationship is introduced. This is useful for models that don't natively handle categorical variables, especially when you need to avoid imposing any artificial hierarchy.

### Convert `Continent` to numeric

For the purposes of this exercise, suppose that the investment group has specified that they want to give more weight to continents with fewer unicorn companies because they believe this could indicate unrealized market potential. 

**Question: Which type of variable would this make the `Continent` variable in terms of how it would be converted to a numeric data type?**


The Continent variable would be considered an ordinal variable when converted to a numeric data type. This is because there is a hierarchy or ranking associated with the continents based on the number of unicorn companies they have. For example, continents with fewer unicorn companies may be considered less important in the context of this analysis, creating a meaningful order or ranking. Therefore, when assigning numeric values to the Continent column, the order of these numbers would reflect the importance of each continent.

Rank the continents in descending order from the greatest number of unicorn companies to the least.

In [47]:
# Rank the continents by number of unicorn companies

companies['Continent'].value_counts()

Continent
North America    586
Asia             310
Europe           143
South America     21
Oceania            8
Africa             3
Name: count, dtype: int64

Now, create a new column called `Continent Number` that represents the `Continent` column converted to numeric in the order of their number of unicorn companies, where North America is encoded as `1`, through Africa, encoded as `6`.

In [48]:
# Create numeric `Continent Number` column

continent_dict={'North America':1,
                'Asia':2,
                'Europe':3,
                'South America':4,
                'Ocenia':5,
                'Africa':6
                }
companies['Continent Number']=companies['Continent'].replace(continent_dict)
companies.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn,High Valuation,Continent Number
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S...",5,high,2
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,high,1
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China...",10,high,2
3,Stripe,95,2014-01-23,Fintech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG",4,high,1
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita...",6,high,3


### Convert `Country/Region` to numeric

Now, suppose that within a given continent, each company's `Country/Region` is given equal importance. For analytical purposes, you want to convert the values in this column to numeric without creating a large number of dummy columns. Use label encoding of this nominal categorical variable to create a new column called `Country/Region Numeric`, wherein each unique `Country/Region` is assigned its own number. 

In [49]:
# Create `Country/Region Numeric` column
# Create numeric categories for Country/Region

companies['Country/Region Numeric']=companies['Country/Region'].astype('category').cat.codes

cat.codes: Requires conversion to a categorical type first.

LabelEncoder: Does not require conversion to a categorical type. It can directly operate on any column with categorical values.

Dummy Encoding: Does not require conversion to a categorical type. It works directly with the data and creates dummy/one-hot encoded columns.

### Convert `Industry` to numeric

Finally, create dummy variables for the values in the `Industry` column. 

In [50]:
# Convert `Industry` to numeric data
# Create dummy variables with Industry values

industry_encoded = pd.get_dummies(companies['Industry'])


# Combine `companies` DataFrame with new dummy Industry columns

companies = pd.concat([companies, industry_encoded], axis=1)

Display the first few rows of `companies`

In [51]:
companies.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn,High Valuation,Continent Number,Country/Region Numeric,Artificial intelligence,Auto & transportation,Consumer & retail,Cybersecurity,Data management & analytics,E-commerce & direct-to-consumer,Edtech,Fintech,Hardware,Health,Internet software & services,Mobile & telecommunications,Other,"Supply chain, logistics, & delivery",Travel
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S...",5,high,2,9,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,high,1,44,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China...",10,high,2,9,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3,Stripe,95,2014-01-23,Fintech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG",4,high,1,44,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita...",6,high,3,38,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False


**Question: Which categorical encoding approach did you use for each variable? Why?**

For encoding the categorical variables:

- Continent: Manual encoding was used to convert the continents into numeric values. This approach established an ordinal relationship based on the perceived hierarchy of importance (e.g., fewer unicorns imply lesser importance). The continents were not first converted into categories, but were directly mapped to numeric values.

- Country/Region: The .cat.codes method was used to encode the Country/Region variable. This method requires the column to be converted into a category type first, which was done here. It created numeric labels for each unique category, but whether this encoding is treated as ordinal depends on the machine learning model used, as it doesn’t inherently establish order.

- Industry: Dummy encoding was used here to create binary columns for each unique category in the Industry column. Since the industries were considered to have no inherent order, this encoding ensures no ordinal relationship is implied. The resulting dummy-encoded columns were concatenated with the original dataframe.

**Question: How does label encoding change the data?**


Label encoding changes the data by assigning each category a unique number, converting qualitative categorical data into numerical format. This transformation makes the data suitable for machine learning models, which typically require numerical inputs. Label encoding also simplifies data visualization and analysis, as numerical data can be more easily represented in charts and graphs. However, it's important to be cautious with label encoding for nominal data, as it might unintentionally introduce an ordinal relationship if the categories are interpreted as having a meaningful order.

**Question: What are the benefits of label encoding?**


Label encoding has several benefits:

- Compatibility with Machine Learning Models: Many machine learning algorithms require numeric inputs, and label encoding transforms categorical variables into numerical values, making the data compatible with such models.

- Data Processing: Label encoding simplifies the processing of categorical data, as machine learning models can perform mathematical operations more easily on numerical data.

- Efficiency: It is a simple and computationally efficient technique, especially when the number of categories is not too large.

- Easy Visualization: Converted numerical values allow for easier plotting and interpretation, as categorical data is transformed into a form that can be visualized in graphs or charts.

However, for nominal data, care must be taken to ensure that the numeric encoding doesn't imply an unintended ordinal relationship.

**Question: What are the disadvantages of label encoding?**


Label encoding has the following disadvantages:

- Unintended Ordinality: If label encoding is applied to nominal (non-ordered) categories, it can introduce a false sense of ordinality. This means that the model might interpret the encoded numbers as having some order or ranking, which may not actually exist between the categories.

- Difficulty in Interpretation: After label encoding, the categories are represented by numbers, which can make it harder to directly interpret the meaning of the values. For example, "red", "green", and "blue" might be encoded as 1, 2, and 3, but it becomes unclear what these numbers represent without referencing the original category labels.

- Impact on Algorithms: Some machine learning algorithms (like linear regression) may treat encoded numerical values as continuous variables, which could be misleading if the data is nominal. For example, assigning "1" to "North" and "2" to "South" may incorrectly imply a numeric relationship that doesn't exist between the two.

- Scalability: Label encoding can become problematic when dealing with a large number of unique categories, as it still assigns a unique number to each category, which can increase computational complexity.

## Conclusion

**What are some key takeaways that you learned during this lab?**

Key Takeaways from this Lab:

- Input Validation: I learned the importance of input validation in ensuring that data is accurate, complete, and free of errors. This is essential for achieving high-quality analysis and results.

- Trial and Error Process: I also discovered that input validation requires an iterative process of identifying issues and choosing the most effective methods for fixing them.

- Label Encoding vs. Dummy Encoding: The lab highlighted the benefits and drawbacks of label encoding and dummy/one-hot encoding. Both methods have their place depending on the type of data and the analysis being performed.

- Decision-Making in Encoding: I understood that the decision to use label encoding or dummy encoding should be based on the specific context and nature of the categorical data at hand.

- Data Cleaning and Validation: Through this lab, I gained hands-on experience in cleaning and validating datasets, including handling missing or inconsistent data, checking for duplicates, and encoding categorical variables for machine learning or further analysis.


**Reference**

[Bhat, M.A. *Unicorn Companies*](https://www.kaggle.com/datasets/mysarahmadbhat/unicorn-companies)

